### Постановка задачи
Заполним отсутствующие значения по погоде интерполяционными данными.

Посчитаем модель линейной регрессии по первому зданию и найдем ее точность.

Данные:
* http://video.ittensive.com/machine-learning/ashrae/building_metadata.csv.gz
* http://video.ittensive.com/machine-learning/ashrae/weather_train.csv.gz
* http://video.ittensive.com/machine-learning/ashrae/train.0.csv.gz
Соревнование: https://www.kaggle.com/c/ashrae-energy-prediction/

© ITtensive, 2020

### Подключение библиотек

In [1]:
import pandas as pd
import numpy as np
from scipy.interpolate import interp1d
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

### Загрузка данных

In [9]:
buildings = pd.read_csv("http://video.ittensive.com/machine-learning/ashrae/building_metadata.csv.gz")
weather = pd.read_csv("http://video.ittensive.com/machine-learning/ashrae/weather_train.csv.gz")
energy = pd.read_csv("http://video.ittensive.com/machine-learning/ashrae/train.0.csv.gz")

### Отсечение здания 0 и отсутствующих значений

In [10]:
energy = energy[(energy["building_id"]==0)]
print (energy)

          building_id  meter            timestamp  meter_reading
0                   0      0  2016-01-01 00:00:00          0.000
1388                0      0  2016-01-01 01:00:00          0.000
2773                0      0  2016-01-01 02:00:00          0.000
4160                0      0  2016-01-01 03:00:00          0.000
5548                0      0  2016-01-01 04:00:00          0.000
...               ...    ...                  ...            ...
12053891            0      0  2016-12-31 19:00:00        232.071
12055295            0      0  2016-12-31 20:00:00        189.069
12056699            0      0  2016-12-31 21:00:00        169.958
12058102            0      0  2016-12-31 22:00:00        169.958
12059506            0      0  2016-12-31 23:00:00        175.418

[8784 rows x 4 columns]


### Объединение данных

In [11]:
energy = pd.merge(left=energy, right=buildings, how="left",
                   left_on="building_id", right_on="building_id")
energy = energy.set_index(["timestamp", "site_id"])
weather = weather.set_index(["timestamp", "site_id"])
energy = pd.merge(left=energy, right=weather, how="left",
                  left_index=True, right_index=True)
energy.reset_index(inplace=True)
energy = energy.drop(columns=["meter", "site_id", "floor_count"], axis=1)
print (energy)
del buildings
del weather
print (energy.info())

                timestamp  building_id  meter_reading primary_use  \
0     2016-01-01 00:00:00            0          0.000   Education   
1     2016-01-01 01:00:00            0          0.000   Education   
2     2016-01-01 02:00:00            0          0.000   Education   
3     2016-01-01 03:00:00            0          0.000   Education   
4     2016-01-01 04:00:00            0          0.000   Education   
...                   ...          ...            ...         ...   
8779  2016-12-31 19:00:00            0        232.071   Education   
8780  2016-12-31 20:00:00            0        189.069   Education   
8781  2016-12-31 21:00:00            0        169.958   Education   
8782  2016-12-31 22:00:00            0        169.958   Education   
8783  2016-12-31 23:00:00            0        175.418   Education   

      square_feet  year_built  air_temperature  cloud_coverage  \
0            7432      2008.0             25.0             6.0   
1            7432      2008.0          

### Оптимизация памяти

In [12]:
def reduce_mem_usage (df):
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if str(col_type)[:5] == "float":
            c_min = df[col].min()
            c_max = df[col].max()
            if c_min > np.finfo("f2").min and c_max < np.finfo("f2").max:
                df[col] = df[col].astype(np.float16)
            elif c_min > np.finfo("f4").min and c_max < np.finfo("f4").max:
                df[col] = df[col].astype(np.float32)
            else:
                df[col] = df[col].astype(np.float64)
        elif str(col_type)[:3] == "int":
            c_min = df[col].min()
            c_max = df[col].max()
            if c_min > np.iinfo("i1").min and c_max < np.iinfo("i1").max:
                df[col] = df[col].astype(np.int8)
            elif c_min > np.iinfo("i2").min and c_max < np.iinfo("i2").max:
                df[col] = df[col].astype(np.int16)
            elif c_min > np.iinfo("i4").min and c_max < np.iinfo("i4").max:
                df[col] = df[col].astype(np.int32)
            elif c_min > np.iinfo("i8").min and c_max < np.iinfo("i8").max:
                df[col] = df[col].astype(np.int64)
        elif col == "timestamp":
            df[col] = pd.to_datetime(df[col])
        elif str(col_type)[:8] != "datetime":
            df[col] = df[col].astype("category")
    end_mem = df.memory_usage().sum() / 1024**2
    print('Потребление памяти меньше на', round(start_mem - end_mem, 2), 'Мб (минус', round(100 * (start_mem - end_mem) / start_mem, 1), '%)')
    return df

In [6]:
energy = reduce_mem_usage(energy)

Потребление памяти меньше на 0.62 Мб (минус 71.1 %)


### Интерполяция данных

In [13]:
energy["precip_depth_1_hr"] = energy["precip_depth_1_hr"].apply(lambda x:x if x>0 else 0)
interpolate_columns = ["air_temperature", "dew_temperature",
                       "cloud_coverage", "wind_speed",
                       "precip_depth_1_hr", "sea_level_pressure"]
for col in interpolate_columns:
    energy[col] = energy[col].interpolate(limit_direction='both',
                            kind='cubic')
print(energy[col])

0       1019.7
1       1020.2
2       1020.2
3       1020.1
4       1020.0
         ...  
8779    1021.7
8780    1021.0
8781    1021.1
8782    1021.1
8783    1021.1
Name: sea_level_pressure, Length: 8784, dtype: float64


### Проверка качества интерполяции

In [8]:
pd.set_option('use_inf_as_na', True)
for col in interpolate_columns:
    print (col, "Inf+NaN:", energy[col].isnull().sum())

air_temperature Inf+NaN: 0
dew_temperature Inf+NaN: 0
cloud_coverage Inf+NaN: 0
wind_speed Inf+NaN: 0
precip_depth_1_hr Inf+NaN: 0
sea_level_pressure Inf+NaN: 0


### Разделение данных

In [10]:
energy_train, energy_test = train_test_split(energy[energy["meter_reading"]>0], test_size=0.2)
print (energy_train.head())

               timestamp  building_id  meter_reading primary_use  square_feet  \
3611 2016-05-30 11:00:00            0        178.125   Education         7432   
7225 2016-10-28 01:00:00            0        206.875   Education         7432   
8600 2016-12-24 08:00:00            0        235.500   Education         7432   
7326 2016-11-01 06:00:00            0        225.250   Education         7432   
7535 2016-11-09 23:00:00            0        219.750   Education         7432   

      year_built  air_temperature  cloud_coverage  dew_temperature  \
3611      2008.0        21.703125        4.000000        20.000000   
7225      2008.0        23.296875        8.000000        16.703125   
8600      2008.0        17.203125        0.000000        15.601562   
7326      2008.0        19.406250        0.000000        17.203125   
7535      2008.0        22.203125        7.667969        16.093750   

      precip_depth_1_hr  sea_level_pressure  wind_direction  wind_speed  
3611              

### Линейная регрессия

In [11]:
regression_columns = ["meter_reading", "air_temperature",
        "dew_temperature", "cloud_coverage", "wind_speed",
        "precip_depth_1_hr", "sea_level_pressure"]

energy_train_lr = pd.DataFrame(energy_train,
    columns=regression_columns)
y = energy_train_lr["meter_reading"]
x = energy_train_lr.drop(labels=["meter_reading"], axis=1)
model = LinearRegression().fit(x, y)
print (model.coef_, model.intercept_)

[ 2.55872086  3.7609389  -2.51010446 -1.69090167  0.1696868  -0.95011603] 1082.8112359757624


### Предсказание и оценка модели

In [12]:
def calculate_model (x):
    lr = np.sum([x[col] * model.coef_[i] for i,col in enumerate(regression_columns[1:])])
    lr += model.intercept_
    x["meter_reading_lr_q"] = (np.log(1 + x.meter_reading) -
                               np.log(1 + lr))**2
    return x

energy_test = energy_test.apply(calculate_model,
                                    axis=1, result_type="expand")
energy_test_lr_rmsle = np.sqrt(energy_test["meter_reading_lr_q"].sum() / len(energy_test))
print ("Качество линейной регрессии:", energy_test_lr_rmsle, round(energy_test_lr_rmsle, 1))

Качество линейной регрессии: 0.2067176002979107 0.2
